# 프랑스 정부 부채 정책 분석 및 예측
# France Government Debt Policy Analysis and Prediction

이 노트북은 5단계 분석 워크플로우를 따릅니다:
1. **데이터 전처리** (Data Preprocessing)
2. **모델 A**: 현상 유지 예측 - Prophet (Baseline)
3. **모델 B**: 긴축 정책 효과 산출 (Policy Effect)
4. **시나리오 합성** (Scenario Synthesis)
5. **최종 시각화** (Final Visualization)

In [ ]:
# 필수 라이브러리 설치 (Install required libraries)
# !pip install prophet pandas numpy matplotlib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
    print("✓ Prophet 사용 가능")
except ImportError:
    PROPHET_AVAILABLE = False
    print("⚠ Prophet 미설치. 설치: pip install prophet")

# 한글 폰트 설정 (Korean font setting)
plt.rcParams['font.family'] = ['DejaVu Sans', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

---
## 🛠️ Step 1. 데이터 전처리 (Data Preprocessing)

- 1990~1999년 데이터 제외, 2000년부터 사용
- 연도별 → 분기별 보간 (Interpolation)
- Train/Test Split: Train(2000-2019), Test(2020-2024)

In [ ]:
# 샘플 데이터 생성 (Create sample debt-to-GDP data)
# 실제 분석에서는 IMF/OECD 데이터로 대체

np.random.seed(42)
years = list(range(1990, 2025))

# 프랑스: 점진적 증가, 2008년 이후 가속, 2020년 급등
france_base = [35 + i * 0.8 for i in range(len(years))]
france = np.array(france_base) + np.random.normal(0, 1.5, len(years))
for i, y in enumerate(years):
    if 2008 <= y <= 2012:
        france[i] += (y - 2007) * 3
    elif y > 2012:
        france[i] += 15
    if y == 2020:
        france[i] += 15
    elif y == 2021:
        france[i] += 12
    elif y >= 2022:
        france[i] += 10

# 포르투갈: 위기 시 급등, 2014년 이후 긴축으로 감소
portugal_base = [50 + i * 1.0 for i in range(len(years))]
portugal = np.array(portugal_base) + np.random.normal(0, 2, len(years))
for i, y in enumerate(years):
    if 2008 <= y <= 2014:
        portugal[i] += (y - 2007) * 5
    elif y > 2014:
        portugal[i] += 35 - (y - 2014) * 2.5
    if y == 2020:
        portugal[i] += 15
    elif y == 2021:
        portugal[i] += 10
    elif y >= 2022:
        portugal[i] += 5

# 아일랜드: 은행 위기로 급등 후 강력한 회복
ireland_base = [30 + i * 0.5 for i in range(len(years))]
ireland = np.array(ireland_base) + np.random.normal(0, 2, len(years))
for i, y in enumerate(years):
    if 2008 <= y <= 2012:
        ireland[i] += (y - 2007) * 10
    elif 2012 < y <= 2014:
        ireland[i] += 50
    elif y > 2014:
        ireland[i] += 50 - (y - 2014) * 5
    if y == 2020:
        ireland[i] += 10
    elif y == 2021:
        ireland[i] += 5

raw_data = pd.DataFrame({
    'Year': years,
    'France': np.clip(france, 30, 130),
    'Portugal': np.clip(portugal, 45, 135),
    'Ireland': np.clip(ireland, 25, 125)
})

print("원본 데이터 (1990-2024):")
print(raw_data.head(10))

In [ ]:
# 2000년 이후 데이터만 필터링
data_filtered = raw_data[raw_data['Year'] >= 2000].copy()
print(f"필터링 후 데이터: {len(data_filtered)} rows (2000-2024)")
print(data_filtered.head())

In [ ]:
# 분기별 보간 (Quarterly Interpolation)
quarterly_data = []
countries = ['France', 'Portugal', 'Ireland']

for i in range(len(data_filtered) - 1):
    year = data_filtered.iloc[i]['Year']
    for q in range(4):
        row = {'Year': year + q * 0.25, 'Quarter': f"{int(year)}-Q{q+1}"}
        for country in countries:
            start_val = data_filtered.iloc[i][country]
            end_val = data_filtered.iloc[i + 1][country]
            row[country] = start_val + (end_val - start_val) * (q / 4)
        quarterly_data.append(row)

# 마지막 연도 추가
last_year = data_filtered.iloc[-1]['Year']
for q in range(4):
    row = {'Year': last_year + q * 0.25, 'Quarter': f"{int(last_year)}-Q{q+1}"}
    for country in countries:
        row[country] = data_filtered.iloc[-1][country]
    quarterly_data.append(row)

processed_data = pd.DataFrame(quarterly_data)
print(f"\n분기별 데이터: {len(processed_data)} rows")
print(processed_data.head(10))

In [ ]:
# Train/Test Split
train_data = processed_data[processed_data['Year'] <= 2019].copy()
test_data = processed_data[processed_data['Year'] >= 2020].copy()

print(f"Train 데이터 (2000-2019): {len(train_data)} rows")
print(f"Test 데이터 (2020-2024): {len(test_data)} rows")

---
## 🤖 Step 2. 모델 A: 프랑스 현상 유지 예측 (Baseline)

- 알고리즘: Facebook Prophet
- 입력: 프랑스 GDP 대비 정부 부채 비율 (2000-2024)
- 출력: 2025-2029년 예측

In [ ]:
if PROPHET_AVAILABLE:
    # Prophet 데이터 준비
    prophet_data = processed_data[['Quarter', 'France']].copy()
    
    # 분기 문자열을 datetime으로 변환
    def quarter_to_date(q_str):
        parts = q_str.split('-Q')
        year = int(parts[0])
        quarter = int(parts[1])
        month = (quarter - 1) * 3 + 1
        return pd.Timestamp(year=year, month=month, day=1)
    
    prophet_data['ds'] = prophet_data['Quarter'].apply(quarter_to_date)
    prophet_data['y'] = prophet_data['France']
    prophet_data = prophet_data[['ds', 'y']]
    
    print("Prophet 입력 데이터:")
    print(prophet_data.head())
else:
    print("Prophet 미설치 - 간단한 선형 추세 모델 사용")

In [ ]:
if PROPHET_AVAILABLE:
    # Prophet 모델 학습
    model = Prophet(
        yearly_seasonality=False,
        weekly_seasonality=False,
        daily_seasonality=False,
        changepoint_prior_scale=0.05
    )
    model.fit(prophet_data)
    
    # 향후 5년 (20분기) 예측
    future = model.make_future_dataframe(periods=20, freq='QS')
    forecast = model.predict(future)
    
    print("\nProphet 예측 완료!")
    print(f"예측 기간: {forecast['ds'].min()} ~ {forecast['ds'].max()}")
else:
    # 대안: 선형 추세 예측
    from scipy import stats
    
    x = processed_data['Year'].values
    y = processed_data['France'].values
    slope, intercept, _, _, _ = stats.linregress(x, y)
    
    future_years = np.arange(2000, 2030.25, 0.25)
    forecast = pd.DataFrame({
        'ds': pd.to_datetime([f"{int(y)}-{int((y % 1) * 12) + 1:02d}-01" for y in future_years]),
        'yhat': intercept + slope * future_years,
        'yhat_lower': intercept + slope * future_years - 5,
        'yhat_upper': intercept + slope * future_years + 5
    })
    print("선형 추세 예측 완료!")

In [ ]:
# 예측 결과 확인
forecast['Year'] = forecast['ds'].dt.year + (forecast['ds'].dt.month - 1) / 12

baseline_forecast = forecast[['ds', 'Year', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
print("\n2025-2029년 예측:")
print(baseline_forecast[baseline_forecast['Year'] >= 2025].head(20))

---
## 📉 Step 3. 모델 B: 긴축 정책 효과 산출 (Policy Effect)

- 이벤트 기준점: 2011년 (포르투갈/아일랜드 구제금융 및 긴축 시작)
- Before (위기): 2008-2010년 (부채 급증 기울기)
- After (긴축): 2012-2015년 (부채 감소 기울기)
- 효과(Delta) = 두 국가의 기울기 변화 평균

In [ ]:
# 기울기 계산 함수
def calculate_slope(data, country, start_year, end_year):
    """지정된 기간의 연평균 변화율 계산"""
    period_data = data[(data['Year'] >= start_year) & (data['Year'] <= end_year)]
    values = period_data[country].values
    years = period_data['Year'].values
    
    if len(values) > 1:
        slope = (values[-1] - values[0]) / (years[-1] - years[0])
    else:
        slope = 0
    return slope

# 포르투갈 기울기 변화
pt_before = calculate_slope(raw_data, 'Portugal', 2008, 2010)
pt_after = calculate_slope(raw_data, 'Portugal', 2012, 2015)
pt_change = pt_after - pt_before

# 아일랜드 기울기 변화
ie_before = calculate_slope(raw_data, 'Ireland', 2008, 2010)
ie_after = calculate_slope(raw_data, 'Ireland', 2012, 2015)
ie_change = ie_after - ie_before

# 평균 효과
delta = (pt_change + ie_change) / 2

print("📊 긴축 정책 효과 분석")
print("=" * 50)
print(f"\n포르투갈:")
print(f"  Before (2008-2010) 기울기: {pt_before:+.2f}%p/년")
print(f"  After (2012-2015) 기울기: {pt_after:+.2f}%p/년")
print(f"  기울기 변화: {pt_change:+.2f}%p/년")

print(f"\n아일랜드:")
print(f"  Before (2008-2010) 기울기: {ie_before:+.2f}%p/년")
print(f"  After (2012-2015) 기울기: {ie_after:+.2f}%p/년")
print(f"  기울기 변화: {ie_change:+.2f}%p/년")

print(f"\n⭐ 최종 효과 (Delta): {delta:+.2f}%p/년")
print(f"   (긴축 정책 적용 시 매년 {abs(delta):.2f}%p씩 부채 비율 감소 효과)")

---
## 🔮 Step 4. 시나리오 합성 (Scenario Synthesis)

- 2025년부터 매년 Delta만큼 누적 적용
- Scenario_Value_t = Baseline_Value_t + (Delta * 경과년수)

In [ ]:
# 시나리오 합성
scenario_start = 2025
scenario_end = 2029

# 2025년 이후 예측 데이터
scenario_forecast = baseline_forecast[
    (baseline_forecast['Year'] >= scenario_start) & 
    (baseline_forecast['Year'] <= scenario_end + 1)
].copy()

# 경과 년수 계산
scenario_forecast['Years_Elapsed'] = scenario_forecast['Year'] - scenario_start + 1

# 시나리오 예측값 계산
scenario_forecast['Baseline_Pred'] = scenario_forecast['yhat']
scenario_forecast['Scenario_Pred'] = scenario_forecast['yhat'] + (delta * scenario_forecast['Years_Elapsed'])

print("시나리오 합성 결과:")
print(scenario_forecast[['Year', 'Baseline_Pred', 'Scenario_Pred', 'Years_Elapsed']].to_string(index=False))

In [ ]:
# 연도별 요약 데이터프레임
yearly_summary = scenario_forecast.groupby(scenario_forecast['Year'].astype(int)).agg({
    'Baseline_Pred': 'mean',
    'Scenario_Pred': 'mean'
}).reset_index()

yearly_summary['Difference'] = yearly_summary['Baseline_Pred'] - yearly_summary['Scenario_Pred']

print("\n📊 연도별 시나리오 비교:")
print(yearly_summary.to_string(index=False))

---
## 📊 Step 5. 최종 시각화 (Final Visualization)

- X축: 2015-2029년
- Y축: GDP 대비 부채 비율 (%)
- 빨간선: 현상 유지 (Baseline)
- 파란선: 긴축 성공 (Scenario)
- 검은점: 실제 데이터

In [ ]:
# 최종 시각화
fig, ax = plt.subplots(figsize=(14, 8))

# 표시 범위
display_start = 2015
display_end = 2029

# 1. 실제 데이터 (검은 점)
hist_data = raw_data[(raw_data['Year'] >= display_start) & (raw_data['Year'] <= 2024)]
ax.scatter(hist_data['Year'], hist_data['France'], 
          color='black', s=100, zorder=5, label='실제 데이터 (Historical)', edgecolors='white', linewidth=1)

# 2. Baseline 예측 (빨간 선)
baseline_display = baseline_forecast[
    (baseline_forecast['Year'] >= display_start) & 
    (baseline_forecast['Year'] <= display_end)
]
ax.plot(baseline_display['Year'], baseline_display['yhat'],
       color='red', linewidth=3, linestyle='-',
       label='현상 유지 (Baseline)', zorder=3)

# Baseline 신뢰구간
ax.fill_between(baseline_display['Year'],
               baseline_display['yhat_lower'],
               baseline_display['yhat_upper'],
               color='red', alpha=0.15)

# 3. Scenario 예측 (파란 선)
scenario_display = scenario_forecast[scenario_forecast['Year'] >= 2025]
ax.plot(scenario_display['Year'], scenario_display['Scenario_Pred'],
       color='blue', linewidth=3, linestyle='-',
       label='긴축 정책 적용 (Austerity)', zorder=4)

# 4. 스타일링
ax.set_xlabel('연도 (Year)', fontsize=14, fontweight='bold')
ax.set_ylabel('GDP 대비 정부 부채 비율 (%)\n(Debt-to-GDP Ratio)', fontsize=14, fontweight='bold')
ax.set_title('프랑스 정부 부채 예측: 현상 유지 vs 긴축 정책\n'
            'France Government Debt Forecast: Baseline vs Austerity Policy',
            fontsize=16, fontweight='bold', pad=20)

ax.legend(loc='upper left', fontsize=12, framealpha=0.95)
ax.grid(True, alpha=0.3, linestyle='--')

# 축 범위
ax.set_xlim(display_start - 0.5, display_end + 0.5)

# 예측 시작점 표시
ax.axvline(x=2024.5, color='gray', linestyle=':', linewidth=2, alpha=0.7)
ax.text(2024.6, ax.get_ylim()[0] + 3, '예측 시작\n(Forecast Start)', 
       fontsize=10, alpha=0.8, fontweight='bold')

plt.tight_layout()

# PNG 저장 (투명 배경)
plt.savefig('debt_policy_forecast.png', dpi=200, bbox_inches='tight', 
           transparent=True, facecolor='none')
print("✓ 그래프 저장 완료: debt_policy_forecast.png (투명 배경)")

plt.show()

---
## 📋 최종 결과 요약 (Summary)

### 분석 결과
1. **현상 유지 시나리오**: 프랑스 부채가 계속 증가
2. **긴축 정책 시나리오**: 포르투갈/아일랜드의 성공 사례를 적용하면 부채 증가 속도 감소

### 정책 시사점
- 포르투갈과 아일랜드의 긴축 정책은 부채 증가율을 효과적으로 낮춤
- 프랑스도 유사한 정책 도입 시 2029년까지 상당한 부채 감소 가능

In [ ]:
# 최종 데이터프레임 출력
print("\n" + "=" * 60)
print("📊 최종 시나리오 비교 데이터")
print("=" * 60)

final_results = yearly_summary.copy()
final_results.columns = ['Year', 'Baseline (현상유지)', 'Scenario (긴축정책)', 'Difference (개선효과)']
print(final_results.to_string(index=False))

print(f"\n⭐ 2029년 예상 개선 효과: {final_results['Difference (개선효과)'].iloc[-1]:.1f}%p")